In [24]:
import sys
sys.path.insert(0, '..')
# sys.path.insert(1, '../python/')
sys.path.insert(1, 'python/')

import ROOT
import root_numpy as rnp
# import python.l1THistos as histos
import math
import uuid
import pandas as pd

from drawingTools import *

# ROOT.enableJSVis()
ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()

In [25]:
# %load python/drawingTools.py

In [26]:
# %load samples.py
import ROOT
import pandas as pd



samples = [
    HistoFile('doubleele_flat1to100_PU200_eg', 'PU200', 'v147D', 'ref'),
    HistoFile('doubleele_flat1to100_PU200_eg', 'PU200', 'v148Bt', 'vertex-emu'),
    HistoFile('doubleele_flat1to100_PU200_eg', 'PU200', 'v148C.67B', 'track-emu'),
    HistoFile('doubleele_flat1to100_PU200_eg', 'PU200', 'v148C.67A', 'track-emu bitwise'),
    HistoFile('doubleele_flat1to100_PU200_eg', 'PU200', 'v150.69A', 'track-emu bitwise (fix)'),
    
    HistoFile('doubleele_flat1to100_PU200_eg', 'PU200', 'v151A.80A', 'ele-12.3'),
    HistoFile('doubleele_flat1to100_PU200_eg', 'PU200', 'v151A.80B', 'ele-12.3.tp'),
    HistoFile('doubleele_flat1to100_PU200_eg', 'PU200', 'v151A.71A', 'ele-11.1'),
    HistoFile('doubleele_flat1to100_PU200_eg', 'PU200', 'v151A.72A', 'ele-11.1.pf'),
    HistoFile('doubleele_flat1to100_PU200_eg', 'PU200', 'v151A.69Ai', 'test'),


]


In [27]:
for hf in samples:
    hf.print_primitives()


<HistoFile ../plots/histos_doubleele_flat1to100_PU200_eg_v147D.root, ref>
--- HistoClass: 
['EGHistos', 'EGResoHistos', 'HistoSetEff']
--- TPs: 
[   'tkEleEB',
    'PFNFtkEmEB',
    'PFNFtkEmEE',
    'tkEleEE',
    'EgEE',
    'PFNFtkEleEB',
    'tkEmEB',
    'PFNFtkEleEE',
    'EgEB',
    'tkEmEE']
--- TP selections:
{   'EgEB': [   'LooseTkIDPt20',
                'LooseTkIDPt10',
                'Pt10',
                'all',
                'Pt20',
                'LooseTkID'],
    'EgEE': [   'EGq5Pt10',
                'EGq5Pt30',
                'EGq4Pt10',
                'EGq4Pt20',
                'EGq4Pt30',
                'EGq4',
                'EGq5Pt20',
                'EGq5'],
    'PFNFtkEleEB': [   'LooseTkIDPt10',
                       'all',
                       'LooseTkIDPt20',
                       'Pt10',
                       'Pt20',
                       'LooseTkID'],
    'PFNFtkEleEE': [   'EGq1Pt10',
                       'EGq2Pt20',
                 

--- HistoClass: 
['EGHistos', 'EGResoHistos', 'HistoSetEff', 'DecTkHistos', 'DecTkResoHistos']
--- TPs: 
[   'tkEleEB',
    'PFNFtkEmEB',
    'PFNFtkEmEE',
    'tkEleEE',
    'PFNFtkEleEB',
    'tkEmEB',
    'PFNFtkEleEE',
    'tkEmEE',
    'PFDecTk']
--- TP selections:
{   'PFDecTk': ['Pt10', 'Pt5', 'Pt20', 'all'],
    'PFNFtkEleEB': [   'LooseTkIDPt10',
                       'all',
                       'LooseTkIDPt20',
                       'Pt10',
                       'Pt20',
                       'LooseTkID'],
    'PFNFtkEleEE': [   'EGq1Pt10',
                       'EGq2Pt20',
                       'EGq2',
                       'EGq1Pt20',
                       'EGq2Pt10',
                       'EGq1'],
    'PFNFtkEmEB': [   'LooseTkID',
                      'Pt20',
                      'Pt10',
                      'LooseTkIDPt20',
                      'all',
                      'LooseTkIDPt10'],
    'PFNFtkEmEE': [   'EGq1',
                      'EGq2Pt10',
   

In [28]:
import python.collections as collections
import python.selections as selections

labels_dict = {}

evm = collections.EventManager()
labels_dict.update(evm.get_labels())
selm = selections.SelectionManager()
labels_dict.update(selm.get_labels())



# #  ==== labels ===============================================
# tp_labels = histo_primitives[['data', 'data_label']].drop_duplicates().set_index('data').T.to_dict('records')[0]
# tp_selection_labels = histo_primitives[['data_sel', 'data_sel_label']].drop_duplicates().set_index('data_sel').T.to_dict('records')[0]
# gen_selection_labels = histo_primitives[['gen_sel', 'gen_sel_label']].drop_duplicates().set_index('gen_sel').T.to_dict('records')[0]






In [29]:
%%time

import python.l1THistos as histos

hplot = HPlot(samples, labels_dict)

hplot.create_histo_proxies(histos.HistoSetEff)
hplot.create_histo_proxies(histos.EGHistos)
hplot.create_histo_proxies(histos.TkEleHistos)
hplot.create_histo_proxies(histos.TkEmHistos)

    
# for tp in tps:
#     hplot.cache_histo(classtype=histos.HistoSetEff, 
#                       samples=samples,
#                       pus=[],
#                       tps=[tp],
#                       tp_sels=histo_primitives[histo_primitives.data == tp].data_sel.unique(),
#                       gen_sels=gen_select[tp])



--- EGHistos
# of plots: 2176
# of primitives: 272
--- EGResoHistos
# of plots: 2448
# of primitives: 272
--- GenParticleHistos
# of plots: 3264
# of primitives: 272


/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append({'sample': sample.type,
/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append({'sample': sample.type,
/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append({'sample': sample.type,
/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: T

/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append({'sample': sample.type,
/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append({'sample': sample.type,
/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append({'sample': sample.type,
/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: T

/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append({'sample': sample.type,
/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append({'sample': sample.type,
/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append({'sample': sample.type,
/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: T

/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append({'sample': sample.type,
/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append({'sample': sample.type,
/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append({'sample': sample.type,
/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: T

...

/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append({'sample': sample.type,
/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append({'sample': sample.type,
/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append({'sample': sample.type,
/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: T

/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append({'sample': sample.type,
/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append({'sample': sample.type,
/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append({'sample': sample.type,
/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: T

/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append({'sample': sample.type,
/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append({'sample': sample.type,
/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append({'sample': sample.type,
/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: T

--- EGHistos
# of plots: 192
# of primitives: 24
--- EGResoHistos
# of plots: 240
# of primitives: 24
--- GenParticleHistos
# of plots: 288
# of primitives: 24
--- EGHistos
# of plots: 2176
# of primitives: 272
--- EGResoHistos
# of plots: 2448
# of primitives: 272
--- GenParticleHistos
# of plots: 3264
# of primitives: 272
*** ERROR: No entry for class: <class 'python.l1THistos.TkEleHistos'> in the primtive index!
--- EGHistos
# of plots: 2176
# of primitives: 272
--- EGResoHistos
# of plots: 2448
# of primitives: 272
--- GenParticleHistos
# of plots: 3264
# of primitives: 272
*** ERROR: No entry for class: <class 'python.l1THistos.TkEmHistos'> in the primtive index!
CPU times: user 19.9 s, sys: 1.59 s, total: 21.5 s
Wall time: 21.6 s


/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plot-drawing-tools/python/drawingTools.py:876: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append({'sample': sample.type,


In [30]:
hplot.data[hplot.data.tp == 'PFtkEmEB'].gen_sel.unique()

array([], dtype=object)

In [31]:
for smp in samples:
    smp.print_primitives()


<HistoFile ../plots/histos_doubleele_flat1to100_PU200_eg_v147D.root, ref>
--- HistoClass: 
['EGHistos', 'EGResoHistos', 'HistoSetEff']
--- TPs: 
[   'tkEleEB',
    'PFNFtkEmEB',
    'PFNFtkEmEE',
    'tkEleEE',
    'EgEE',
    'PFNFtkEleEB',
    'tkEmEB',
    'PFNFtkEleEE',
    'EgEB',
    'tkEmEE']
--- TP selections:
{   'EgEB': [   'LooseTkIDPt20',
                'LooseTkIDPt10',
                'Pt10',
                'all',
                'Pt20',
                'LooseTkID'],
    'EgEE': [   'EGq5Pt10',
                'EGq5Pt30',
                'EGq4Pt10',
                'EGq4Pt20',
                'EGq4Pt30',
                'EGq4',
                'EGq5Pt20',
                'EGq5'],
    'PFNFtkEleEB': [   'LooseTkIDPt10',
                       'all',
                       'LooseTkIDPt20',
                       'Pt10',
                       'Pt20',
                       'LooseTkID'],
    'PFNFtkEleEE': [   'EGq1Pt10',
                       'EGq2Pt20',
                 

--- HistoClass: 
['EGHistos', 'EGResoHistos', 'HistoSetEff']
--- TPs: 
[   'tkEleEB',
    'PFNFtkEmEB',
    'PFNFtkEmEE',
    'tkEleEE',
    'EgEE',
    'PFNFtkEleEB',
    'tkEmEB',
    'PFNFtkEleEE',
    'EgEB',
    'tkEmEE']
--- TP selections:
{   'EgEB': [   'LooseTkIDPt20',
                'LooseTkIDPt10',
                'Pt10',
                'all',
                'Pt20',
                'LooseTkID'],
    'EgEE': [   'EGq5Pt10',
                'EGq5Pt30',
                'EGq4Pt10',
                'EGq4Pt20',
                'EGq4Pt30',
                'EGq4',
                'EGq5Pt20',
                'EGq5'],
    'PFNFtkEleEB': [   'LooseTkIDPt10',
                       'all',
                       'LooseTkIDPt20',
                       'Pt10',
                       'Pt20',
                       'LooseTkID'],
    'PFNFtkEleEE': [   'EGq1Pt10',
                       'EGq2Pt20',
                       'EGq2',
                       'EGq1Pt20',
                       'E

### Gen level particle distribution

In [32]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['ref', 'ele-11.1', 'test'], ['PU200'], ['tkEleEB', 'TkEleEB', 'PFNFtkEleEB'], 'all', 'GEN')            
draw([his.h_den.h_pt for his in hsets], labels, norm=True, text=text)
draw([his.h_den.h_eta for his in hsets], labels, norm=True, text=text, v_lines=[-2.8,-2.4,-1.7,-1.52,2.8,2.4,1.7,1.52])
draw([his.h_den.h_fBrem for his in hsets], labels, norm=True, text=text)
draw([his.h_den.h_reachedEE for his in hsets], labels, norm=True, text=text)


In [33]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['ref', 'ele-11.1', 'test'], ['PU200'], ['tkEleEB', 'TkEleEB', 'PFNFtkEleEB'], 'all', 'GEN')            
draw([his.h_num.h_pt for his in hsets], labels, norm=True, text=text)
draw([his.h_num.h_eta for his in hsets], labels, norm=True, text=text, v_lines=[-2.8,-2.4,-1.7,-1.52,2.8,2.4,1.7,1.52])
draw([his.h_num.h_fBrem for his in hsets], labels, norm=True, text=text)
draw([his.h_num.h_reachedEE for his in hsets], labels, norm=True, text=text)


## EG standalone

In [ ]:
hsets, labels, text = hplot.get_histo(histos.EGHistos, 'dele-V11', ['PU200'], ['EgEE', 'PFEgEE', 'PFNFEgEE'], 'EGq5', 'nomatch')            
draw([his.h_pt for his in hsets], labels, norm=True, text=text, options='hist', logy=True)
draw([his.h_eta for his in hsets], labels, logy=True, norm=True, text=text, v_lines=[-3,-2.5,-1.52,1.52,2.5,3], options='hist')


In [ ]:
# %load python/drawingTools.py


In [ ]:
hsets, labels, text = hplot.get_histo(histos.EGHistos, 'dele-V11', ['PU200'], ['PFNFEgEE', 'PFEgEE', 'EgEE'], ['EGq5', 'EGq4',], 'GENEtaBCD')            
print(hsets[0].h_pt.GetName())
draw([his.h_pt for his in hsets], labels, norm=False, text=text, options='hist', logy=False)

for his in hsets:
    if his.h_eta.GetNbinsX() == 100:
        his.h_eta.Rebin(2)

draw([his.h_eta for his in hsets], labels, norm=False, text=text, v_lines=[-2.8,-2.4,-1.7,-1.52,2.8,2.4,1.7,1.52])


In [ ]:
TP: tkEmEE
['EGq5' 'EGq5Pt20' 'EGq4Pt20' 'EGq4' 'EGq5Pt30' 'EGq4Pt10' 'EGq5Pt10'
 'EGq4Pt30']
['GENEePt15' 'GENEeEtaBC' 'GENEePt30' 'GENEe']

In [16]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.3, 0.5)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.EGHistos, 
    ['track-emu bitwise'], 
    ['PU200'], 
    ['tkEleEE'], 
    ['EGq4'], 'GENEePt15', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_tkIso for his in hsets], labels=['sim (TDR) EE'])


# hsets, labels, text = hplot.get_histo(
#     histos.HistoSetEff, 
#     ['track-emu bitwise'], 
#     ['PU200'], 
#     ['tkEmEB'], 
#     ['all'], 'GENPt15', debug=False)
# # for hset in hsets:
# #     hset.computeEff(rebin=2)
# dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=['sim (TDR) EB'])


# hsets, labels, text = hplot.get_histo(
#     histos.EGHistos, 
#     ['ref'], 
#     ['PU200'], 
#     ['PFNFtkEleEE'], 
#     ['EGq1'], 'GENEePt15', debug=False)
# # for hset in hsets:
# #     hset.computeEff(rebin=2)
# dm.addHistos([his.h_tkIso for his in hsets], labels=['EG emu EE'])


hsets, labels, text = hplot.get_histo(
    histos.EGHistos, 
    ['vertex-emu'], 
    ['PU200'], 
    ['PFNFtkEleEE'], 
    ['EGq1'], 'GENEePt15', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_tkIso for his in hsets], labels=['vertex-emu EE'])

hsets, labels, text = hplot.get_histo(
    histos.EGHistos, 
    ['track-emu'], 
    ['PU200'], 
    ['PFNFtkEleEE'], 
    ['EGq1'], 'GENEePt15', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_tkIso for his in hsets], labels=['track-emu EE'])

hsets, labels, text = hplot.get_histo(
    histos.EGHistos, 
    ['track-emu bitwise'], 
    ['PU200'], 
    ['PFNFtkEleEE'], 
    ['EGq1'], 'GENEePt15', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_tkIso for his in hsets], labels=['track-emu bitwise EE'])


hsets, labels, text = hplot.get_histo(
    histos.EGHistos, 
    ['track-emu bitwise (fix)'], 
    ['PU200'], 
    ['PFNFtkEleEE'], 
    ['EGq1'], 'GENEePt15', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_tkIso for his in hsets], labels=['track-emu bitwise EE (fix)'])



# hsets, labels, text = hplot.get_histo(
#     histos.HistoSetEff, 
#     ['vertex-emu'], 
#     ['PU200'], 
#     ['PFNFtkEmEB'], 
#     ['all'], 'GENPt15', debug=False)
# # for hset in hsets:
# #     hset.computeEff(rebin=2)
# dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=['vertex-emu EB'])



# hsets, labels, text = hplot.get_histo(
#     histos.HistoSetEff, 
#     ['track-emu'], 
#     ['PU200'], 
#     ['PFNFtkEmEE'], 
#     ['EGq1'], 'GENEePt15', debug=False)
# # for hset in hsets:
# #     hset.computeEff(rebin=2)
# dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=['track-emu EE'])

# hsets, labels, text = hplot.get_histo(
#     histos.HistoSetEff, 
#     ['track-emu'], 
#     ['PU200'], 
#     ['PFNFtkEmEB'], 
#     ['all'], 'GENPt15', debug=False)
# # for hset in hsets:
# #     hset.computeEff(rebin=2)
# dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=['track-emu EB'])



# hsets, labels, text = hplot.get_histo(
#     histos.HistoSetEff, 
#     ['track-emu bitwise'], 
#     ['PU200'], 
#     ['PFNFtkEmEE'], 
#     ['EGq1'], 'GENEePt15', debug=False)
# # for hset in hsets:
# #     hset.computeEff(rebin=2)
# dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=['track-emu bitwise EE'])

# hsets, labels, text = hplot.get_histo(
#     histos.HistoSetEff, 
#     ['track-emu bitwise'], 
#     ['PU200'], 
#     ['PFNFtkEmEB'], 
#     ['all'], 'GENPt15', debug=False)
# # for hset in hsets:
# #     hset.computeEff(rebin=2)
# dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=['track-emu bitwise EB'])



# dm.addRatioHisto(0,1)
# dm.addRatioHisto(2,1)
dm.addRatioHisto(3,1)
dm.addRatioHisto(4,1)


dm.draw(text='EG, TkEle, p_{T}^{GEN}>15GeV', 
#         x_min=0., x_max=3.2, 
       y_min=0.5, y_max=1E5, 
#       v_lines=[1.52, 2.4],
       do_ratio=True,
       y_min_ratio=0.8,
       y_max_ratio=1.2,
       y_log=True)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





0.0 2.0
0.0 2.0
0.0 2.0


In [ ]:
newconfig=rleg_config
newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.3, 0.5)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(
    histos.EGHistos, 
    ['track-emu bitwise'], 
    ['PU200'], 
    ['tkEmEE'], 
    ['EGq4'], 'GENEePt15', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_tkIsoPV for his in hsets], labels=['sim (TDR) EE'])


# hsets, labels, text = hplot.get_histo(
#     histos.HistoSetEff, 
#     ['track-emu bitwise'], 
#     ['PU200'], 
#     ['tkEmEB'], 
#     ['all'], 'GENPt15', debug=False)
# # for hset in hsets:
# #     hset.computeEff(rebin=2)
# dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=['sim (TDR) EB'])


# hsets, labels, text = hplot.get_histo(
#     histos.EGHistos, 
#     ['ref'], 
#     ['PU200'], 
#     ['PFNFtkEleEE'], 
#     ['EGq1'], 'GENEePt15', debug=False)
# # for hset in hsets:
# #     hset.computeEff(rebin=2)
# dm.addHistos([his.h_tkIso for his in hsets], labels=['EG emu EE'])


hsets, labels, text = hplot.get_histo(
    histos.EGHistos, 
    ['vertex-emu'], 
    ['PU200'], 
    ['PFNFtkEmEE'], 
    ['EGq1'], 'GENEePt15', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_tkIsoPV for his in hsets], labels=['vertex-emu EE'])

hsets, labels, text = hplot.get_histo(
    histos.EGHistos, 
    ['track-emu'], 
    ['PU200'], 
    ['PFNFtkEmEE'], 
    ['EGq1'], 'GENEePt15', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_tkIsoPV for his in hsets], labels=['track-emu EE'])

hsets, labels, text = hplot.get_histo(
    histos.EGHistos, 
    ['track-emu bitwise'], 
    ['PU200'], 
    ['PFNFtkEmEE'], 
    ['EGq1'], 'GENEePt15', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
dm.addHistos([his.h_tkIsoPV for his in hsets], labels=['track-emu bitwise EE'])



# hsets, labels, text = hplot.get_histo(
#     histos.HistoSetEff, 
#     ['vertex-emu'], 
#     ['PU200'], 
#     ['PFNFtkEmEB'], 
#     ['all'], 'GENPt15', debug=False)
# # for hset in hsets:
# #     hset.computeEff(rebin=2)
# dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=['vertex-emu EB'])



# hsets, labels, text = hplot.get_histo(
#     histos.HistoSetEff, 
#     ['track-emu'], 
#     ['PU200'], 
#     ['PFNFtkEmEE'], 
#     ['EGq1'], 'GENEePt15', debug=False)
# # for hset in hsets:
# #     hset.computeEff(rebin=2)
# dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=['track-emu EE'])

# hsets, labels, text = hplot.get_histo(
#     histos.HistoSetEff, 
#     ['track-emu'], 
#     ['PU200'], 
#     ['PFNFtkEmEB'], 
#     ['all'], 'GENPt15', debug=False)
# # for hset in hsets:
# #     hset.computeEff(rebin=2)
# dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=['track-emu EB'])



# hsets, labels, text = hplot.get_histo(
#     histos.HistoSetEff, 
#     ['track-emu bitwise'], 
#     ['PU200'], 
#     ['PFNFtkEmEE'], 
#     ['EGq1'], 'GENEePt15', debug=False)
# # for hset in hsets:
# #     hset.computeEff(rebin=2)
# dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=['track-emu bitwise EE'])

# hsets, labels, text = hplot.get_histo(
#     histos.HistoSetEff, 
#     ['track-emu bitwise'], 
#     ['PU200'], 
#     ['PFNFtkEmEB'], 
#     ['all'], 'GENPt15', debug=False)
# # for hset in hsets:
# #     hset.computeEff(rebin=2)
# dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], labels=['track-emu bitwise EB'])



# dm.addRatioHisto(0,1)
# dm.addRatioHisto(2,1)
# dm.addRatioHisto(3,1)


dm.draw(text='EG, TkEm, p_{T}^{GEN}>15GeV', 
#         x_min=0., x_max=3.2, 
       y_min=0.5, y_max=1E5, 
#       v_lines=[1.52, 2.4],
#        do_ratio=True,
       y_min_ratio=0.8,
       y_max_ratio=1.2,
       y_log=True)
# dm.write(name='eg_TDRvsSummer20_matchig_eff')





In [ ]:
hsets, labels, text = hplot.get_histo(histos.TkEleHistos, 'dele-V11', ['PU200'], ['tkEleEE', 'PFtkEleEE', 'PFNFtkEleEE'], 'EGq5', 'nomatch')            
draw([his.h_pt for his in hsets], labels, norm=False, text=text, options='hist', logy=False)
for his in hsets:
    if his.h_eta.GetNbinsX() == 100:
        his.h_eta.Rebin(2)

draw([his.h_eta for his in hsets], labels, norm=True, text=text, v_lines=[-2.4,-1.7,-1.52,2.4,1.7,1.52])
draw([his.h_tkIso for his in hsets], labels, norm=False, text=text, logy=True, options='hist')
draw([his.h_pfIso for his in hsets], labels, norm=False, text=text, logy=True, options='hist')
draw([his.h_tkchi2 for his in hsets], labels, norm=False, text=text, logy=True, options='hist')
# draw([his.h_tkz0 for his in hsets], labels, norm=False, text=text, logy=True, options='hist')


In [ ]:
hsets, labels, text = hplot.get_histo(histos.EGHistos, 'dele-V11', ['PU200'], ['tkEleEE', 'PFtkEleEE'], 'EGq5', 'GENEtaBC')            
draw([his.h_pt for his in hsets], labels, norm=False, text=text, options='hist', logy=False)
for his in hsets:
    if his.h_eta.GetNbinsX() == 100:
        his.h_eta.Rebin(2)

draw([his.h_eta for his in hsets], labels, norm=True, text=text, v_lines=[-2.4,-1.7,-1.52,2.4,1.7,1.52])
draw([his.h_tkIso for his in hsets], labels, norm=False, text=text, logy=True, options='hist')
draw([his.h_pfIso for his in hsets], labels, norm=False, text=text, logy=True, options='hist')
# draw([his.h_tkchi2 for his in hsets], labels, norm=False, text=text, logy=True, options='hist')
# draw([his.h_tkz0 for his in hsets], labels, norm=False, text=text, logy=True, options='hist')


In [ ]:
hsets, labels, text = hplot.get_histo(histos.EGHistos, 'dele-V11', ['PU200'], ['tkEleEB', 'PFtkEleEB', 'PFNFtkEleEB'], 'LooseTkID', 'GENEtaF')            
draw([his.h_pt for his in hsets], labels, norm=False, text=text, options='hist', logy=False)
for his in hsets:
    if his.h_eta.GetNbinsX() == 100:
        his.h_eta.Rebin(2)

draw([his.h_eta for his in hsets], labels, norm=True, text=text, v_lines=[-2.4,-1.7,-1.52,2.4,1.7,1.52])
draw([his.h_tkIso for his in hsets], labels, norm=False, text=text, logy=True, options='hist')
draw([his.h_pfIso for his in hsets], labels, norm=False, text=text, logy=True, options='hist')
# draw([his.h_tkchi2 for his in hsets], labels, norm=False, text=text, logy=True, options='hist')
# draw([his.h_tkz0 for his in hsets], labels, norm=False, text=text, logy=True, options='hist')


In [ ]:
hplot.data[(hplot.data['sample'] == 'dele-V11') & (hplot.data.pu == 'PU200') & (hplot.data.tp == 'tkEleEE') & (hplot.data.tp_sel == 'EGq5') & (hplot.data.gen_sel == 'GENEtaBC')]

In [ ]:
hsets, labels, text = hplot.get_histo(histos.TkEmHistos, 'dele-V11', ['PU200'], ['PFtkEmEB', 'tkEmEB'], 'all', 'nomatch')            
draw([his.h_pt for his in hsets], labels, norm=False, text=text, options='hist', logy=False)
for his in hsets:
    if his.h_eta.GetNbinsX() == 100:
        his.h_eta.Rebin(2)
    
draw([his.h_eta for his in hsets], labels, norm=False, text=text, v_lines=[-2.4,-1.7,-1.52,2.4,1.7,1.52])
draw([his.h_tkIso for his in hsets], labels, norm=False, text=text, logy=True, options='hist')
draw([his.h_pfIso for his in hsets], labels, norm=False, text=text, logy=True, options='hist')
draw([his.h_tkIsoPV for his in hsets], labels, norm=False, text=text, logy=True, options='hist')
draw([his.h_pfIsoPV for his in hsets], labels, norm=False, text=text, logy=True, options='hist')
print (hsets[0].h_eta.GetEntries())
print (hsets[1].h_eta.GetEntries())

In [ ]:
hsets, labels, text = hplot.get_histo(histos.EGHistos, 'dele-V11', ['PU200'], ['PFtkEmEE', 'tkEmEE'], 'EGq5', 'GENEtaBC')            
draw([his.h_pt for his in hsets], labels, norm=False, text=text, options='hist', logy=False)
for his in hsets:
    if his.h_eta.GetNbinsX() == 100:
        his.h_eta.Rebin(2)

draw([his.h_eta for his in hsets], labels, norm=False, text=text, v_lines=[-2.4,-1.7,-1.52,2.4,1.7,1.52])
draw([his.h_tkIso for his in hsets], labels, norm=False, text=text, logy=True, options='hist')
draw([his.h_pfIso for his in hsets], labels, norm=False, text=text, logy=True, options='hist')


In [ ]:
hsets, labels, text = hplot.get_histo(histos.EGHistos, 'dele-V11', ['PU200'], ['PFtkEleEE', 'tkEleEE'], 'EGq5', 'GENEtaBC')            
draw([his.h_pt for his in hsets], labels, norm=False, text=text, options='hist', logy=False)
for his in hsets:
    if his.h_eta.GetNbinsX() == 100:
        his.h_eta.Rebin(2)

draw([his.h_eta for his in hsets], labels, norm=False, text=text, v_lines=[-2.4,-1.7,-1.52,2.4,1.7,1.52])
draw([his.h_tkIso for his in hsets], labels, norm=False, text=text, logy=True, options='hist')
draw([his.h_pfIso for his in hsets], labels, norm=False, text=text, logy=True, options='hist')


In [ ]:
newconfig=tdr_config
newconfig.legend_size=(0.5, 0.2)
newconfig.legend_position=(0.4, 0.2)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)
hsets, labels, text = hplot.get_histo(histos.TkEmHistos, 'dele-V11', ['PU200'], ['PFtkEmEB'], 'Pt10', 'nomatch')            
dm.addHistos([his.h_tkIsoPV for his in hsets], ['tkIso (PV)'])

dm.addHistos([his.h_pfIsoPV for his in hsets], ['pfIso (PV)'])
print(hsets[0].h_tkIsoPV.GetBinContent(0))
print(hsets[0].h_pfIsoPV.GetBinContent(-1))

# hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['dele-NM'], ['PU200'], ['EGALL'], 'Pt25LooseTkID', 'GENPt35', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
# dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], ['EG calo-only (CMSSW 11.1)'])


dm.draw(text=text, y_log=True, options='hist')

In [ ]:
newconfig=tdr_config
newconfig.legend_size=(0.5, 0.2)
newconfig.legend_position=(0.4, 0.2)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)
hsets, labels, text = hplot.get_histo(histos.TkEmHistos, 'dele-V11', ['PU200'], ['PFtkEmEB'], 'all', 'nomatch')            
dm.addHistos([his.h_tkIso for his in hsets], ['tkIso'])
dm.addHistos([his.h_pfIso for his in hsets], ['pfIso'])

# hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['dele-NM'], ['PU200'], ['EGALL'], 'Pt25LooseTkID', 'GENPt35', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
# dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], ['EG calo-only (CMSSW 11.1)'])


dm.draw(text=text, y_log=True, options='hist')

In [ ]:
newconfig=tdr_config
newconfig.legend_size=(0.5, 0.2)
newconfig.legend_position=(0.4, 0.2)
newconfig.marker_styles.append(10)
dm = DrawMachine(newconfig)
hsets, labels, text = hplot.get_histo(histos.TkEleHistos, 'dele-V11', ['PU200'], ['PFtkEleEB'], 'Pt10', 'nomatch')            
dm.addHistos([his.h_tkIso for his in hsets], ['tkIso'])
dm.addHistos([his.h_pfIso for his in hsets], ['pfIso'])

# hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['dele-NM'], ['PU200'], ['EGALL'], 'Pt25LooseTkID', 'GENPt35', debug=False)
# for hset in hsets:
#     hset.computeEff(rebin=2)
# dm.addHistos([his.h_eff.h_abseta.CreateGraph() for his in hsets], ['EG calo-only (CMSSW 11.1)'])


dm.draw(text=text, y_log=True, options='hist')

In [ ]:
hsets, labels, text = hplot.get_histo(histos.TkEleHistos, 'dele-V11', ['PU200'], ['tkEleEB', 'PFtkEleEB'], 'all', 'nomatch')            
draw([his.h_pt for his in hsets], labels, norm=False, text=text, options='hist', logy=False)
for his in hsets:
    if his.h_eta.GetNbinsX() == 100:
        his.h_eta.Rebin(2)

draw([his.h_eta for his in hsets], labels, norm=False, text=text, v_lines=[-2.4,-1.7,-1.52,2.4,1.7,1.52])
draw([his.h_tkIso for his in hsets], labels, norm=False, text=text, logy=True, options='hist')
draw([his.h_pfIso for his in hsets], labels, norm=False, text=text, logy=True, options='hist')


In [ ]:
hsets, labels, text = hplot.get_histo(histos.EGHistos, 'dele-V11', ['PU200'], ['tkEmEE', 'PFtkEmEE'], 'EGq5', 'GENPt15')            
draw([his.h_pt for his in hsets], labels, norm=True, text=text, options='hist', logy=False)
for his in hsets:
    if his.h_eta.GetNbinsX() == 100:
        his.h_eta.Rebin(2)

draw([his.h_eta for his in hsets], labels, norm=False, text=text, v_lines=[-2.4,-1.7,-1.52,2.4,1.7,1.52])
draw([his.h_tkIso for his in hsets], labels, norm=True, text=text, logy=True, options='hist')
# draw([his.h_pfIso for his in hsets], labels, norm=False, text=text, logy=True, options='hist')


In [ ]:
hsets, labels, text = hplot.get_histo(histos.EGHistos, 'dele-V11', ['PU200'], ['tkEmEE', 'PFtkEmEE'], 'EGq5Pt10', 'GENEtaBC')            
draw([his.h_pt for his in hsets], labels, norm=False, text=text, options='hist', logy=False)
for his in hsets:
    if his.h_eta.GetNbinsX() == 100:
        his.h_eta.Rebin(2)

draw([his.h_eta for his in hsets], labels, norm=False, text=text, v_lines=[-2.4,-1.7,-1.52,2.4,1.7,1.52])
draw([his.h_tkIso for his in hsets], labels, norm=False, text=text, logy=True, options='hist')
# draw([his.h_pfIso for his in hsets], labels, norm=False, text=text, logy=True, options='hist')


In [ ]:
hsets, labels, text = hplot.get_histo(histos.EGHistos, 'dele-V11', ['PU200'], ['tkEleEE', 'PFtkEleEE'], 'EGq5', 'GENPt15')            
draw([his.h_pt for his in hsets], labels, norm=False, text=text, options='hist', logy=False)
draw([his.h_eta for his in hsets], labels, norm=False, text=text, v_lines=[-2.4,-1.7,-1.52,2.4,1.7,1.52])
draw([his.h_tkIso for his in hsets], labels, norm=False, text=text, logy=True, options='hist')
# draw([his.h_pfIso for his in hsets], labels, norm=False, text=text, logy=True, options='hist')


In [ ]:
hsets, labels, text = hplot.get_histo(histos.EGHistos, 'dele-V11', ['PU200'], ['tkEleEE', 'PFtkEleEE'], 'EGq5', 'GENEtaBC')            
draw([his.h_pt for his in hsets], labels, norm=False, text=text, options='hist', logy=False)
draw([his.h_eta for his in hsets], labels, norm=False, text=text, v_lines=[-2.4,-1.7,-1.52,2.4,1.7,1.52])
draw([his.h_tkIso for his in hsets], labels, norm=False, text=text, logy=True, options='hist')
# draw([his.h_pfIso for his in hsets], labels, norm=False, text=text, logy=True, options='hist')


In [ ]:
hsets, labels, text = hplot.get_histo(histos.EGHistos, 'dele-V11', ['PU200'], ['tkEmEE', 'PFtkEmEE', 'PFNFtkEmEE'], 'EGq5', 'GENEtaBC')            
draw([his.h_pt for his in hsets], labels, norm=False, text=text, options='hist', logy=False)
draw([his.h_eta for his in hsets], labels, norm=False, text=text, v_lines=[-2.4,-1.7,-1.52,2.4,1.7,1.52])
draw([his.h_tkIso for his in hsets], labels, norm=False, text=text, logy=True, options='hist')
draw([his.h_pfIso for his in hsets], labels, norm=False, text=text, logy=True, options='hist')
draw([his.h_tkIsoPV for his in hsets], labels, norm=False, text=text, logy=True, options='hist')
draw([his.h_pfIsoPV for his in hsets], labels, norm=False, text=text, logy=True, options='hist')
